In [5]:
pip install flask flask-sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from flask import Flask, render_template, request
from flask_sqlalchemy import SQLAlchemy
import nest_asyncio

In [8]:
nest_asyncio.apply()

In [9]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///news_database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

In [10]:
db = SQLAlchemy(app)

In [11]:
class Article(db.Model):
    __tablename__ = 'articles'
    id = db.Column(db.Integer, primary_key=True)
    header = db.Column(db.String(255), nullable=False)
    full_text = db.Column(db.Text, nullable=False)

In [12]:
class ArticleMetadata(db.Model):
    __tablename__ = 'article_metadata'
    id = db.Column(db.Integer, primary_key=True)
    article_id = db.Column(db.Integer, db.ForeignKey('articles.id'), nullable=False)
    pub_date = db.Column(db.String(50), nullable=False)

In [13]:
class Category(db.Model):
    __tablename__ = 'categories'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

In [14]:
class ArticleCategory(db.Model):
    __tablename__ = 'article_categories'
    id = db.Column(db.Integer, primary_key=True)
    article_id = db.Column(db.Integer, db.ForeignKey('articles.id'), nullable=False)
    category_id = db.Column(db.Integer, db.ForeignKey('categories.id'), nullable=False)

In [15]:
def create_app():
    with app.app_context():
        db.create_all()  # Создаёт все таблицы, если они ещё не существуют
    return app

In [16]:
df = pd.read_csv("news.csv")

In [17]:
with app.app_context():

    # Удаляем все предыдущие записи из таблиц
    db.session.query(ArticleCategory).delete()
    db.session.query(ArticleMetadata).delete()
    db.session.query(Category).delete()
    db.session.query(Article).delete()
    db.session.commit()
    
    for _, row in df.iterrows():
        # 1. Добавляем статью в таблицу Article
        article = Article(header=row['header'], full_text=row['full_text'])
        db.session.add(article)
        db.session.commit()

        # 2. Добавляем метаданные статьи в ArticleMetadata
        metadata = ArticleMetadata(
            article_id=article.id,
            pub_date=row['pub_date']
        )
        db.session.add(metadata)

        # 3. Добавляем тег в Category (если его ещё нет)
        tag_name = row['tag'].strip()
        category = Category.query.filter_by(name=tag_name).first()
        if not category:
            category = Category(name=tag_name)
            db.session.add(category)
            db.session.commit()

        # 4. Связываем статью с категорией в ArticleCategory
        article_category = ArticleCategory(article_id=article.id, category_id=category.id)
        db.session.add(article_category)

    db.session.commit()  # Сохраняем все изменения

In [18]:
@app.route("/")
def index():
    return render_template("index.html")

In [19]:
@app.route("/add", methods=["GET", "POST"])
def add_news():
    if request.method == "POST":
        pub_date = request.form["pub_date"]
        tag = request.form["tag"]
        header = request.form["header"]
        full_text = request.form["full_text"]

        # Добавляем новость в базу данных
        new_article = Article(header=header, full_text=full_text)
        db.session.add(new_article)
        db.session.commit()

        metadata = ArticleMetadata(article_id=new_article.id, pub_date=pub_date)
        db.session.add(metadata)

        category = Category.query.filter_by(name=tag).first()
        if not category:
            category = Category(name=tag)
            db.session.add(category)
            db.session.commit()

        article_category = ArticleCategory(article_id=new_article.id, category_id=category.id)
        db.session.add(article_category)
        db.session.commit()

        return render_template("success.html", header=header)
    return render_template("add_news.html")

In [20]:
@app.route("/statistics")
def statistics():
    # 1. Общее количество статей
    total_articles = Article.query.count()
    
    # 2. Самый длинный заголовок статьи
    longest_title = (
    db.session.query(Article.header)
    .order_by(db.func.length(Article.header).desc())
    .first()
)
    
    # 3. Дата с наибольшим количеством опубликованных статей
    most_popular_date = (
        db.session.query(
            ArticleMetadata.pub_date.label("pub_date"),
            db.func.count(ArticleMetadata.article_id).label("article_count")
        )
        .group_by(ArticleMetadata.pub_date)
        .order_by(db.func.count(ArticleMetadata.article_id).desc())
        .first()
    )

    # 4. Общее количество категорий
    total_categories = Category.query.count()

    # 5. Топ-5 самых частотных тегов
    top_tags = (
        db.session.query(
            Category.name.label("tag_name"),
            db.func.count(ArticleCategory.category_id).label("tag_count")
        )
        .join(ArticleCategory, Category.id == ArticleCategory.category_id)
        .group_by(Category.name)
        .order_by(db.func.count(ArticleCategory.category_id).desc())
        .limit(5)
        .all()
    )

    return render_template(
        "statistics.html",
        total_articles=total_articles,
        longest_title=longest_title[0] if longest_title else "Нет данных",
        most_popular_date=most_popular_date,
        total_categories=total_categories,
        top_tags=top_tags,
    )

In [ ]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Dec/2024 21:32:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2024 21:32:34] "GET /static/images/db_schema.png HTTP/1.1" 304 -
127.0.0.1 - - [20/Dec/2024 21:32:38] "GET /statistics HTTP/1.1" 200 -
